## Solving using Dwave

In [1]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod

In [2]:
items_values = [8, 47, 10, 5, 16]
items_weight = [3, 11, 14, 19, 5]
maximum_weight = 26

In [3]:
n_items = len(items_values)
Q = -np.diag(items_values)

In [4]:
lambd = 10 
N = round(np.ceil(np.log2(maximum_weight + 1)))
print(N)
weights = items_weight + [2**k for k in range(N)]
print(Q)
QT = np.pad(Q, ((0, N), (0, N))) 
print(QT)

for i in range(len(QT)):
    QT[i, i] += lambd * weights[i] * (weights[i] - 2 * maximum_weight)
    for j in range(i + 1, len(QT)):
        QT[i, j] += 2 * lambd * weights[i] * weights[j]
print(QT)

5
[[ -8   0   0   0   0]
 [  0 -47   0   0   0]
 [  0   0 -10   0   0]
 [  0   0   0  -5   0]
 [  0   0   0   0 -16]]
[[ -8   0   0   0   0   0   0   0   0   0]
 [  0 -47   0   0   0   0   0   0   0   0]
 [  0   0 -10   0   0   0   0   0   0   0]
 [  0   0   0  -5   0   0   0   0   0   0]
 [  0   0   0   0 -16   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]
[[-1478   660   840  1140   300    60   120   240   480   960]
 [    0 -4557  3080  4180  1100   220   440   880  1760  3520]
 [    0     0 -5330  5320  1400   280   560  1120  2240  4480]
 [    0     0     0 -6275  1900   380   760  1520  3040  6080]
 [    0     0     0     0 -2366   100   200   400   800  1600]
 [    0     0     0     0     0  -510    40    80   160   320]
 [    0     0     0     0     0     0 -1000   160   320   640]
 [    0   

In [5]:
sampler = EmbeddingComposite(DWaveSampler())

response = sampler.sample_qubo(QT, num_reads=500)
offset = lambd * maximum_weight**2
# print(offset)
best_solution = None
lowest_energy = float('inf')

for datum in response.data():
    sample = datum.sample
    energy = datum.energy
    if energy < lowest_energy:
        lowest_energy = energy
        best_solution = sample

selected_items = [index for index in range(n_items) if best_solution[index] == 1]
print(f"Best solution found:")
print(f"Items: {selected_items}, Energy: {lowest_energy + offset}")


Best solution found:
Items: [0, 1, 4], Energy: -71.0
